![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [165]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

In [166]:
rental_info = pd.read_csv('rental_info.csv')
rental_info.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [167]:
rental_info.dtypes

rental_date          object
return_date          object
amount              float64
release_year        float64
rental_rate         float64
length              float64
replacement_cost    float64
special_features     object
NC-17                 int64
PG                    int64
PG-13                 int64
R                     int64
amount_2            float64
length_2            float64
rental_rate_2       float64
dtype: object

In [168]:
rental_info['rental_date'] = pd.to_datetime(rental_info['rental_date'])
rental_info['return_date'] = pd.to_datetime(rental_info['return_date'])

In [169]:
rental_info.dtypes

rental_date         datetime64[ns, UTC]
return_date         datetime64[ns, UTC]
amount                          float64
release_year                    float64
rental_rate                     float64
length                          float64
replacement_cost                float64
special_features                 object
NC-17                             int64
PG                                int64
PG-13                             int64
R                                 int64
amount_2                        float64
length_2                        float64
rental_rate_2                   float64
dtype: object

In [170]:
rental_info.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [171]:
rental_info['rental_length_days'] = (rental_info['return_date'] - rental_info['rental_date']).dt.days

In [172]:
rental_info.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4


In [173]:
# cleaning rental pandas
rental_info = rental_info.drop(['rental_date', 'return_date'], axis=1)

In [174]:
rental_info.head()

,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days
0,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4


In [175]:
special_features = rental_info['special_features'].unique()

In [176]:
set_special_features = set(
    category.strip('""')
    for special_feature in special_features
    for category in special_feature[1:-1].split(',')
)
set_special_features

{'Behind the Scenes', 'Commentaries', 'Deleted Scenes', 'Trailers'}

In [177]:
rental_info['parsed_features'] = rental_info['special_features'].apply(lambda x: {cat.strip().strip('""') for cat in x[1:-1].split(',')})

In [178]:
rental_info.dtypes

amount                float64
release_year          float64
rental_rate           float64
length                float64
replacement_cost      float64
special_features       object
NC-17                   int64
PG                      int64
PG-13                   int64
R                       int64
amount_2              float64
length_2              float64
rental_rate_2         float64
rental_length_days      int64
parsed_features        object
dtype: object

In [179]:
for special_feature in set_special_features:
    rental_info[special_feature] = rental_info['parsed_features'].apply(lambda x: 1 if special_feature in x else 0)

In [180]:
rental_info = rental_info.drop(['special_features', 'parsed_features'], axis=1)

In [181]:
rental_info.head()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,Trailers,Deleted Scenes,Commentaries,Behind the Scenes
0,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,3,1,0,0,1
1,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,2,1,0,0,1
2,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,7,1,0,0,1
3,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,2,1,0,0,1
4,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,4,1,0,0,1


In [182]:
import matplotlib.pyplot as plt

In [183]:
rental_info.describe()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,Trailers,Deleted Scenes,Commentaries,Behind the Scenes
count,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000
mean,4.217161,2006.885379,2.944101,114.994578,20.224727,0.204842,0.200303,0.223378,0.198726,23.355504,14832.841876,11.389287,4.525944,0.530610,0.497320,0.543156,0.536347
std,2.360383,2.025027,1.649766,40.114715,6.083784,0.403599,0.400239,0.416523,0.399054,23.503164,9393.431996,10.005293,2.635108,0.499078,0.500009,0.498150,0.498693
min,0.990000,2004.000000,0.990000,46.000000,9.990000,0.000000,0.000000,0.000000,0.000000,0.980100,2116.000000,0.980100,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.990000,2005.000000,0.990000,81.000000,14.990000,0.000000,0.000000,0.000000,0.000000,8.940100,6561.000000,0.980100,2.000000,0.000000,0.000000,0.000000,0.000000
50%,3.990000,2007.000000,2.990000,114.000000,20.990000,0.000000,0.000000,0.000000,0.000000,15.920100,12996.000000,8.940100,5.000000,1.000000,0.000000,1.000000,1.000000
75%,4.990000,2009.000000,4.990000,148.000000,25.990000,0.000000,0.000000,0.000000,0.000000,24.900100,21904.000000,24.900100,7.000000,1.000000,1.000000,1.000000,1.000000
max,11.990000,2010.000000,4.990000,185.000000,29.990000,1.000000,1.000000,1.000000,1.000000,143.760100,34225.000000,24.900100,9.000000,1.000000,1.000000,1.000000,1.000000


In [184]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [185]:
X = rental_info.drop(['rental_length_days', 'Trailers', 'Commentaries'], axis=1)
y = rental_info['rental_length_days']

In [186]:
X = X.rename(columns={'Deleted Scenes': 'deleted_scenes', 'Behind the Scenes': 'behind_the_scenes'})

In [187]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [188]:
rfg = RandomForestRegressor(random_state=9)
rf_params = {
    'n_estimators': [50,100],
    'max_depth': [2,3],
    'min_samples_leaf': [50, 100, 150],
    'max_features': [0.7, 0.8, 0.9],
    'ccp_alpha': [0.0000001]
}
gridCV = GridSearchCV(rfg, rf_params, n_jobs=-1, cv=5, scoring='neg_mean_squared_error')

In [189]:
gridCV.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=9), n_jobs=-1,
             param_grid={'ccp_alpha': [1e-07], 'max_depth': [2, 3],
                         'max_features': [0.7, 0.8, 0.9],
                         'min_samples_leaf': [50, 100, 150],
                         'n_estimators': [50, 100]},
             scoring='neg_mean_squared_error')

In [202]:
gridCV.best_params_

{'ccp_alpha': 1e-07,
 'max_depth': 3,
 'max_features': 0.9,
 'min_samples_leaf': 50,
 'n_estimators': 100}

In [203]:
y_pred = gridCV.predict(X_test)

In [204]:
best_mse = mean_squared_error(y_test, y_pred)
best_mse

2.8797197036677513

In [205]:
best_gridcv = gridCV.best_estimator_

In [206]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(base_estimator=best_gridcv, n_estimators=180, random_state=9, learning_rate=0.5)

In [207]:
ada.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=RandomForestRegressor(ccp_alpha=1e-07,
                                                       max_depth=3,
                                                       max_features=0.9,
                                                       min_samples_leaf=50,
                                                       random_state=9),
                  learning_rate=0.5, n_estimators=180, random_state=9)

In [208]:
y_pred_ada = ada.predict(X_test)
mse_ada = mean_squared_error(y_test, y_pred)
mse_ada

2.8797197036677513

# Probando con Gradient Boost

In [196]:
X.head()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,deleted_scenes,behind_the_scenes
0,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
1,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
2,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
3,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
4,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1


In [197]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(random_state=9)
params_gbr = {
    'max_depth': [2,3,4],
    'n_estimators': [200],
    'subsample': [0.9,0.8,0.7],
}
gridGBR = GridSearchCV(gbr, params_gbr, n_jobs=-1, cv=5)

In [198]:
gridGBR.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=9),
             n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4], 'n_estimators': [200],
                         'subsample': [0.9, 0.8, 0.7]})

In [199]:
y_pred = gridGBR.predict(X_test)
best_mse = mean_squared_error(y_test, y_pred)
best_mse

2.1547812876113697

In [200]:
best_model = gridGBR.best_estimator_